<a href="https://colab.research.google.com/github/AprendaPracticando/AnaliticaPythonR1/blob/main/LABS/LAB_12_01_Ausentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LAB 12.01: Tratamiento de datos ausentes**

En este Lab se desea generar, paso a paso, un categórico de intervalo.
Las tareas por realizar son:
1.	Definir la estrategia de manejo de atípicos del caso.
2.	Revisión de existencia de datos ausentes.
3.	Estrategia de eliminación de filas con ausentes. 


In [ ]:
# Datos base
import pandas as pd

tipos_correctos={
    'id_persona':object,
    'clave_sobrevivencia':object,
    'clase_viaje':object
}
personas_titanic_csv='https://raw.githubusercontent.com/AprendaPracticando/AnaliticaPythonR1/main/data/personas_titanic_v5.csv'
titanic = pd.read_csv(personas_titanic_csv,
      dtype=tipos_correctos)

titanic['id_persona']=titanic['id_persona'].str.split('.').str.get(0)
titanic['clave_sobrevivencia']=titanic['clave_sobrevivencia'].str.split('.').str.get(0)
titanic['clase_viaje']=titanic['clase_viaje'].str.split('.').str.get(0)


## **Corrección de MAR usando requeridos indirectos y UDF**

In [ ]:
# Identifica los ausentes del campo sobrevivencia.
titanic[['nombre','clave_sobrevivencia',
         'sobrevivencia','id_bote',
         'id_cuerpo']][(titanic['sobrevivencia'].isnull())]


Como podemos ver, hay tres filas con datos ausentes, pero sabemos también que a partir de **`id_bote`** y **`id_cuerpo`** podemos inferir sobrevivencia:
1.	Si **`id_bote`** no es nulo, quiere decir que la persona se subió a un bote salvavidas, y vivió.
2.	Si **`id_cuerpo`** no es nulo, quiere decir que le asignaron un número de cuerpo, porque murió.

Elaboramos una función definida por el usuario para hacer el cálculo, y actualizamos **`sobrevivencia`** y **`clave_sobrevivencia`**.


In [ ]:
# Función que infiere el valor de sobreviencia a partir de 
# id_bote o id_cuerpo.
def corrige_sobrevivencia(fila):
  # El valor actual de sobrevivencia se asigna a una variable.
  etiqueta=fila['sobrevivencia']
  # Si id_cuerpo no es nulo, la etiqueta pasa a MURIÓ
  if (not pd.isnull(fila['id_cuerpo'])):
    etiqueta='MURIÓ'
  # Si id_bote no es nulo, la etiqueta pasa a VIVIÓ
  if (not pd.isnull(fila['id_bote'])):
    etiqueta='VIVIÓ'
  # Se retorna la etiqueta.
  return etiqueta


In [ ]:
def corrige_clave_sobrevivencia(fila):
  # El valor actual de sobrevivencia se asigna a una variable.
  etiqueta=fila['clave_sobrevivencia']
  # Validamos que la etiqueta no esté vacía o sea nula
  if (not pd.isnull(fila['clave_sobrevivencia'])):
    if (fila['sobrevivencia']=='MURIÓ'):
      etiqueta='0'
    else:
      etiqueta='1'     
  # Se retorna la etiqueta.
  return etiqueta


In [ ]:
# Se aplica la función definida por el usuario, a cada
# fila del DataSet, actualizando sobrevivencia con el 
# valor retornado.
titanic['sobrevivencia']=titanic.apply(corrige_sobrevivencia,axis=1)
titanic['clave_sobrevivencia']=titanic.apply(corrige_clave_sobrevivencia,axis=1)


In [ ]:
# Identifica los ausentes del campo sobrevivencia.
# Sigue habiendo uno, porque no hay información ni en
# cuerpo ni en bote, y no hay otra manera de saber si
# vivió o murió.
# Muestra las columnas nombre, clave_sobrevivencia,
# sobrevivencia, id_bote y id_cuerpo, cuando sobrevivencia
# sea nulo.
titanic[['nombre','clave_sobrevivencia',
         'sobrevivencia','id_bote',
         'id_cuerpo']][(titanic['sobrevivencia'].isnull())]

# Este dato es MNAR (missing not at random),
# pues sistemáticamente, en todos los casos, si tenemos
# bote o cuerpo, podemos inferir sobrevivencia.


## **Corrección de MAR por asignación directa**

In [ ]:
# Identifica los ausentes del campo sexo.
titanic[['nombre','sexo']][(titanic['sexo'].isnull())]


Las filas con el índice 148 y 237 tienen información suficiente para concluir que los pasajeros eran hombres.

La fila con el índice 299 no tiene forma de inferir el sexo de la persona, porque no incluye título, y Alexis es un nombre que aplica tanto para hombres como para mujeres.

Se actualizan los registros que se pueden corregir, de manera específica, usando **`at[]`**.


In [ ]:
# Se le asigna el valor 'HOMBRE' a la columna
# sexo, a las filas con índice 148 y 237, usando
# at[]
titanic.at[148,'sexo']="HOMBRE"
titanic.at[237,'sexo']="HOMBRE"




In [ ]:
# Identifica los ausentes del campo sexo.
titanic[['nombre','sexo']][(titanic['sexo'].isnull())]